In [ ]:
# default_exp models

# models
> a

In [ ]:
#hide
from nbdev import *
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.showdoc import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import networkx as nx
import numpy as np
import typing
from typing import Optional, Tuple, Dict, Callable, Union, Mapping, Sequence, Iterable, Hashable, List, Any
from collections import namedtuple
import sidis
from sidis import *

C:\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [ ]:
import networkm.network
from networkm.network import *

In [ ]:
g=ring(left=True,right=False,loop=False)
g.add_edges_from([(2,2),(3,2),(1,1)])

for i,n in enumerate(g.nodes):
    g.nodes[n]['n']=i
    
for i,e in enumerate(g.edges):
    g.edges[e]['delay']=i
    
G=Network(g,relabel=False)
G

|Node|Predecessors|Successors|
|2   |3, 0, 2     |2, 1      |
|1   |1, 2        |1, 0      |
|0   |1           |2         |
|3   |            |2         |

In [ ]:
print(G.edges(data=True))
print(G.nodes(data=True))

[(2, 2, {'delay': 4}), (2, 1, {'delay': 3}), (1, 1, {'delay': 2}), (1, 0, {'delay': 1}), (0, 2, {'delay': 0}), (3, 2, {'delay': 5})]
[(2, {'n': 2}), (1, {'n': 1}), (0, {'n': 0}), (3, {'n': 3})]


In [ ]:
#export
class NetworkModel(Network):
    '''
    Generic network model class. Has base functionality for incorporating
    a network and simulating its dynamics, as well as executing arbitrary 
    routines. Can pass a network `g`, `node_data`, `edge_data`, and `kwargs`
    which get placed as attributes. `routines` is a list of functions,
    args, kwargs, and optional string attributes, which specify a list
    of functions to execute over the args and kwargs, and whether to set
    the result as an attribute of the model. This allows for arbitrary
    executions of routines. Finally, `derivative`, `integrator`, and
    `visualizer` specify the state-change and plotting functions.
    '''
    def __init__(self,
                 g : Union[None,Network,nx.MultiDiGraph] = None,
                 node_data : Dict = {},
                 edge_data : Dict = {},
                 routines : List[Tuple[callable,tuple,dict,str]] = [(None,(),{},None)],
                 derivative : Optional[callable] = None,
                 integrator : Optional[callable] = None,
                 visualizer : Optional[callable] = None,
                 exclusions : List[str] = ['self'],
                 *args,
                 **kwargs
                ):
        
        self.__dict__.update({k:v for k,v in locals().items() if k not in exclusions})
        
        super().__init__(g,relabel=True,view='out')
        
        self.give(node_data=node_data,edge_data=edge_data)
        
        self.routine(routines)

        
        def give(self,node_data=None,edge_data=None,model_data=None,nodes=None,edges=None):
            self.give_nodes(data=node_data,nodes=nodes,**node_data)
            self.give_edges(data=edge_data,edges=edges,**edge_data)
            self.give_self(data=model_data,**model_data)
            self.node_data(save=True)
            self.edge_data(save=True)

        def derive(self,*args,**kwargs):
            res = self.derivative(*args,**kwargs)
            if res:
                return res
        @timer    
        def integrate(self,*args,**kwargs):
            res = self.integrator(*args,**kwargs)
            if res:
                return res
        
        def visualize(self,*args,**kwargs):
            res = self.visualizer(*args,**kwargs)
            if res:
                return res
        
        def routine(self,routines=None):
            if routines is None:
                routines=self.routines
            res = []
            for tup in routines:
                if len(tup)==3: #return func eval
                    f,args,kwargs = tup
                    try:
                        res += [f(*args,**kwargs)]
                    except:
                        pass
                elif len(tup) == 4: #set eval as str
                    f,args,kwargs,attr = tup
                    try:
                        temp=f(*args,**kwargs)
                        setattr(self,attr,temp)
                    except:
                        pass
            if res:
                return res

In [ ]:
#export
def dde_iter(G : Network,
             delay_key : str = 'delay') -> List[List[Union[tuple,np.ndarray,np.ndarray]]]:
    '''
    Construct a list of iterables used in the calculation
    of a delay-differential-equation (dde) for a network `G`.
    Returns a list-of-lists containing  [(node_start,node_stop), 
    adjacency matrix, delay matrix] for each in-degree group.
    '''
    res=[]
    node_index=0
    for deg,nodes in G.sorting.items():
        if deg==0:
            pass
        else:
            L=len(nodes)
            temp=[]
            temp+=[(node_index,node_index+L)]
            temp+=[np.array([list(G.predecessors(n)) for n in nodes])]
            temp+=[np.array([G.edges[e][delay_key] \
                for e in G.in_edges(nodes,keys=True)]).reshape((len(nodes),deg))]
            res+=[temp]
            node_index+=L
    return res

In [ ]:
g=ring(left=True,right=False,loop=False)
g.add_edges_from([(2,2),(3,2),(1,1)])

for i,n in enumerate(g.nodes):
    g.nodes[n]['n']=i
    
for i,e in enumerate(g.edges):
    g.edges[e]['delay']=i
    
G=Network(g,relabel=True)

In [ ]:
di=dde_iter(G,delay_key='delay')
di

[[(0, 1), array([[0, 2, 3]]), array([[4, 0, 5]])],
 [(1, 2), array([[0, 1]]), array([[3, 2]])],
 [(2, 3), array([[1]]), array([[1]])]]

In [ ]:
#export
def dde_iter(G : Network,
             delay_key : str = 'delay') -> List[List[Union[tuple,np.ndarray,np.ndarray]]]:
    '''
    Construct a list of iterables used in the calculation
    of a delay-differential-equation (dde) for a network `G`.
    Returns a list-of-lists containing  [(node_start,node_stop), 
    adjacency matrix, delay matrix] for each in-degree group.
    '''
    node_bounds=[]
    adj_matrices=[]
    delay_matrices=[]
    node_index=0
    for deg,nodes in G.sorting.items():
        if deg==0:
            pass
        else:
            L=len(nodes)
            node_bounds+=[(node_index,node_index+L)]
            adj_matrices+=[np.array([list(G.predecessors(n)) for n in nodes])]
            delay_matrices+=[np.array([G.edges[e][delay_key] \
                for e in G.in_edges(nodes,keys=True)]).reshape((len(nodes),deg))]
            node_index+=L
    return node_bounds,adj_matrices,delay_matrices

In [ ]:
node_bounds,adj_matrices,delay_matrices=dde_iter(G)

In [ ]:
np.max([np.max(delays) for delays in delay_matrices])

5

In [ ]:
def dde_discrete(t,x,dx,f,tau,delays,epsilon,iterator,sigmoid=np.rint):
    for (n1,n2),in_nodes in iterator:
        y=sigmoid(x[delays,in_nodes])
        dx[n1:n2]=np.array([F(*y[j]) for j,F in enumerate(f[n1:n2])])
    return (-x[t]+dx+epsilon)/tau

def dde(t,x,dx,f,a,tau,delays,epsilon,iterator):
    for (n1,n2),in_nodes,sigmoid in iterator:
        #s=lambda z: (1+np.tanh(a[n1:n2].reshape((n2-n1,1))*(z-0.5)))/2
        y=sigmoid(x[delays,in_nodes])
        dx[n1:n2]=sigmoid(np.array([F(*y[j]) for j,F in enumerate(f[n1:n2])]))
    return (-x[t]+dx+epsilon)/np.choose(np.rint(x[t]).astype(int),tau)

@timer
def euler_dde_hold(x,
                   dx,
                   dt,
                   T,
                   hold,
                   initial_state,
                   deriv,
                   delays,
                   node_bounds,
                   adj_matrices,
                   delay_matrices
                   **kwargs):
    max_delay=np.max([np.max(delays) for delays in delay_matrices])
    delay_matrices=[-delays for delays in delay_matrices]
    max_hold=np.max(hold)
    for t in range(max_delay,max_hold):
        delays+=t
        holds=np.ravel(np.argwhere(hold>=t))
        x[t,holds]=initial_state[holds]
        x[t+1]=x[t]+dt*deriv(t=t,x=x,delays=delays,**kwargs)
    for t in range(max_hold,T-1):
        delays+=t
        x[t+1]=x[t]+dt*deriv(t=t,x=x,delays=delays,**kwargs)